In [52]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns

import string
from tqdm.notebook import tqdm
import matplotlib
from sklearn.metrics import classification_report

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import pymorphy2

from sklearn.decomposition import TruncatedSVD
import matplotlib.patches as mpatches

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

import tika
from tika import parser
import re
import os
from sklearn.preprocessing import LabelEncoder

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\isdom\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\isdom\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\isdom\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\isdom\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [66]:

uploaded_file = parser.from_file("073a0d372820c3c2bffe9ba24a7ed7af.doc")
#uploaded_file['content']

parsed = pd.DataFrame()
parsed.loc[0,'content'] = uploaded_file['content']
'''
bytes_data = uploaded_file.getvalue()
data = parser.from_buffer(bytes_data)
'''


stop_words = set(stopwords.words('russian')) # множество стоп слов

morph = pymorphy2.MorphAnalyzer() # для постановки слова в начальную форму

def lemmatize_words(text):
    '''Функция для лемматизации отдельных слов.'''
    final_text = []
    for i in text.lower().split():
        if i not in stop_words:
            parse = morph.parse(i)[0]
            if ('Abbr' not in parse.tag):
                final_text.append(parse.normal_form)               
    return " ".join(final_text)

def text_preprocessing(text):
    data = text.copy()

    # удаляем пунктуацию
    data['content_punct'] = data['content'].str.translate(str.maketrans('', '', string.punctuation))
    # Приводим к начальной форме
    data['content_punct_lemm'] = data['content_punct'].apply(lemmatize_words)
    
    return data

x_pred=text_preprocessing(parsed)
x_pred=x_pred['content_punct_lemm']
x_pred= pd.DataFrame(x_pred)

import joblib

tf = joblib.load('tf.pkl')
LR_trained_classifier = joblib.load('LR_trained_classifier.pkl')
sample_vec = tf.transform(x_pred)

pr = LR_trained_classifier.predict(sample_vec)
pr1 = LR_trained_classifier.predict_proba(sample_vec)

classes = pd.read_json('classes.json', orient='index')
text_class = classes[0].unique()
predict = text_class[pr]
print(predict)

['Договоры для акселератора/Договоры подряда']


In [57]:
import pickle
list = [1, 2, 3]
pickle.dump(list, 'binary_list.bin')

TypeError: file must have a 'write' attribute

In [67]:
text_class

array(['Договоры для акселератора/Договоры поставки',
       'Договоры для акселератора/Договоры оказания услуг',
       'Договоры для акселератора/Договоры подряда',
       'Договоры для акселератора/Договоры аренды',
       'Договоры для акселератора/Договоры купли-продажи'], dtype=object)

In [69]:
pr

array([2])